In [ ]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Activation, Dense, LSTM, Dropout,BatchNormalization
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.regularizers import l2

In [ ]:
filepath = '/content/drive/MyDrive/shakespeare (1).txt'
with open(filepath, 'rb') as file:text = file.read().decode(encoding='utf-8').lower()

In [ ]:
# text = text[300000:800000]

In [ ]:
text

"first citizen:\r\nbefore we proceed any further, hear me speak.\r\n\r\nall:\r\nspeak, speak.\r\n\r\nfirst citizen:\r\nyou are all resolved rather to die than to famish?\r\n\r\nall:\r\nresolved. resolved.\r\n\r\nfirst citizen:\r\nfirst, you know caius marcius is chief enemy to the people.\r\n\r\nall:\r\nwe know't, we know't.\r\n\r\nfirst citizen:\r\nlet us kill him, and we'll have corn at our own price.\r\nis't a verdict?\r\n\r\nall:\r\nno more talking on't; let it be done: away, away!\r\n\r\nsecond citizen:\r\none word, good citizens.\r\n\r\nfirst citizen:\r\nwe are accounted poor citizens, the patricians good.\r\nwhat authority surfeits on would relieve us: if they\r\nwould yield us but the superfluity, while it were\r\nwholesome, we might guess they relieved us humanely;\r\nbut they think we are too dear: the leanness that\r\nafflicts us, the object of our misery, is as an\r\ninventory to particularise their abundance; our\r\nsufferance is a gain to them let us revenge this with\r\n

In [ ]:
characters = sorted(set(text))
char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

In [ ]:
SEQ_LENGTH = 40
STEP_SIZE = 3

sentences = []
next_char = []

In [ ]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_char.append(text[i + SEQ_LENGTH])

In [ ]:
x = np.zeros((len(sentences), SEQ_LENGTH,
              len(characters)), dtype=np.bool_)
y = np.zeros((len(sentences),
              len(characters)), dtype=np.bool_)

for i, satz in enumerate(sentences):
    for t, char in enumerate(satz):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_char[i]]] = 1


In [ ]:
# approch 1

model = Sequential()
model.add(LSTM(128, return_sequences=True,
               input_shape=(SEQ_LENGTH, len(characters)),
               kernel_regularizer=l2(0.001)))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.001)))
model.add(Dropout(0.2))
model.add(LSTM(32, kernel_regularizer=l2(0.001)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dense(len(characters), activation='softmax'))

learning_rate = ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.9
)
optimizer = RMSprop(learning_rate=learning_rate)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                        │ (None, 40, 128)             │          86,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 40, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 40, 64)              │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 40, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │           8,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 40)                  │          10,280 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 168,104 (656.66 KB)

 Trainable params: 167,592 (654.66 KB)

 Non-trainable params: 512 (2.00 KB)

In [ ]:
#approch 2

# model = Sequential()
# model.add(LSTM(128, return_sequences=True, input_shape=(SEQ_LENGTH, len(characters))))
# model.add(Dropout(0.2))
# model.add(LSTM(64))
# model.add(Dropout(0.2))
# model.add(Dense(len(characters), activation='softmax'))

# optimizer = Adam(learning_rate=0.001)
# model.compile(loss='categorical_crossentropy',
#               optimizer=optimizer,
#               metrics=['accuracy']
# )
# model.summary()

In [ ]:
#approch 3

# model = Sequential()
# model.add(LSTM(128,input_shape=(SEQ_LENGTH,len(characters))))
# model.add(Dense(len(characters)))
# model.add(Activation('softmax'))

# model.compile(loss='categorical_crossentropy',
#               optimizer=RMSprop(learning_rate=0.01),metrics=['accuracy'])

# model.summary()

In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=3)
model.fit(x, y, batch_size=256, epochs=10, callbacks=[early_stopping])

Epoch 1/10
1505/1505 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.3951 - loss: 2.1159
Epoch 2/10
1505/1505 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - accuracy: 0.4011 - loss: 2.0923
Epoch 3/10
1505/1505 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - accuracy: 0.4063 - loss: 2.0746
Epoch 4/10
1505/1505 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.4090 - loss: 2.0567
Epoch 5/10
1505/1505 ━━━━━━━━━━━━━━━━━━━━ 22s 13ms/step - accuracy: 0.4113 - loss: 2.0435
Epoch 6/10
1505/1505 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.4142 - loss: 2.0324
Epoch 7/10
1505/1505 ━━━━━━━━━━━━━━━━━━━━ 19s 13ms/step - accuracy: 0.4172 - loss: 2.0206
Epoch 8/10
1505/1505 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.4202 - loss: 2.0111
Epoch 9/10
1505/1505 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.4203 - loss: 2.0047
Epoch 10/10
1505/1505 ━━━━━━━━━━━━━━━━━━━━ 21s 12ms/step - accuracy: 0.4229 - loss: 1.9923


In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

In [ ]:
print(generate_text(300, 0.2))
print(generate_text(300, 0.4))
print(generate_text(300, 0.5))
print(generate_text(300, 0.6))
print(generate_text(300, 0.9))
print(generate_text(300, 0.1))

your bloods,
currents that spring from the son the shall here be the sonter
the dean the mine the son the meales the shees,
the let the lord the fand and the the some.

kich than
the san what the lite the let my me the the sovere
the some the see the mene the deer the could the shall
and the parter of the sheak the the sented
the
body,
and makes her pew-fellow with other the sornes the senther
the lard the dool the send and me me dailss.

duch manger:
the hent you with the sonted the done seet:
whe have did is the count fe hath thou will and the fantt
so i sir the the me when shall that sealder
the courter the pontell the sich i the chenting and the line

 it, that will free your life,
but fetting suke the sontes, conlest the ree.

gount etal:
i day as the sure thei shear the mean you his shave your sees
the list the reet howe to whose the gate
that with the mene i pallon the hore the though they
the crone the to my wheshed i sigh with shened
the munse the grion the parten,
what t

my 